In [8]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    to be used after 03_real_power_consumption
    it calculates the quality of the approximation and adjust the missing node power consumption
'''

datadir = "/datasets/eurora_data/db1/"
#datadir = "/datasets/eurora_data/db/"
#datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"


infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-06-30')
end_time = pd.to_datetime('2014-11-01')
infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_359350"

## 4 settings BeforeAlina
interval_comment = "BeforeAlina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-07-01')
infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_108074"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']


Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [9]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [10]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [11]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [12]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

drop the jobs out of interval

In [13]:
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs.shape[0]))

jobs_whole_data contains 100003 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
jobs = jobs[pd.to_datetime(jobs['end_time']) <= end_time]
jobs = jobs[pd.to_datetime(jobs['run_start_time']) > start_time]
print("Alina's interval contains {} records".format(jobs.shape[0]))

Alina's interval contains 100003 records


#### merge

In [15]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

108074


### add an approximation of the power consumed on completely missing nodes

In [16]:
#debug
jobs[jobs['good_nodes'] != jobs['n_2_1'] + jobs['n_2_2'] + jobs['n_3_1']].shape[0]

2309

In [17]:
#debug
jobs[jobs['good_nodes'] != jobs['node_req']].shape[0]

2367

In [18]:
jobs_to_adjust = jobs[jobs['good_nodes'] != jobs['node_req']]

In [19]:
for index_job, row_job in jobs_to_adjust.iterrows():
    tot_nodes = row_job['node_req']
    real_pow = row_job['real_pow']
    
    real_pow += (1 - row_job['good_nodes'] / tot_nodes) * real_pow
    
    jobs.at[index_job, 'real_pow'] = real_pow

In [20]:
#debug
#jobs_to_adjust = jobs[jobs['good_nodes'] != jobs['node_req']]
#jobs_to_adjust[jobs_to_adjust['real_pow'] > 200]

### calculate the job pow approximation quality

In [21]:
jobs_0_pow = jobs[jobs['real_pow_quality'] == 0]

In [22]:
237428-122780

114648

In [23]:
(pd.to_datetime(jobs_0_pow['end_time']) - pd.to_datetime(jobs_0_pow['run_start_time'])).mean()

Timedelta('0 days 00:29:32.293341')

In [24]:
jobs['real_pow_quality']  = 1 - ((jobs['job_tot_timepoints'] - jobs['job_timepoints']) / jobs['job_tot_timepoints'])

In [25]:
jobs.sort_values('real_pow_quality')

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
102391,805372.node129,p-H-N-rad.cmd,np_longpar,2014-06-30 23:19:57,2014-06-30 23:34:51,2014-06-30 23:34:58,pstipa00@node129.eurora.cineca.it,1,16,14680064,...,0.0,147.073296,True,0.0,0,0,1,1,0,1
27504,533067.node129,run_SMD,parallel,2014-04-05 21:12:31,2014-04-05 21:12:32,2014-04-05 21:12:37,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,10.767923,True,0.0,0,0,1,1,0,1
27505,533068.node129,run_SMD,parallel,2014-04-05 21:12:37,2014-04-05 21:12:38,2014-04-05 21:12:43,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,11.010326,True,0.0,0,0,1,1,0,1
27506,533069.node129,run_SMD,parallel,2014-04-05 21:12:43,2014-04-05 21:12:44,2014-04-05 21:12:49,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,11.355076,True,0.0,0,0,1,1,0,1
27507,533070.node129,run_SMD,parallel,2014-04-05 21:12:49,2014-04-05 21:12:50,2014-04-05 21:12:55,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,10.789750,True,0.0,0,0,1,1,0,1
27508,533071.node129,run_SMD,parallel,2014-04-05 21:12:55,2014-04-05 21:12:56,2014-04-05 21:13:01,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,11.010954,True,0.0,0,0,1,1,0,1
27509,533072.node129,run_SMD,parallel,2014-04-05 21:13:01,2014-04-05 21:13:02,2014-04-05 21:13:07,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,11.019512,True,0.0,0,0,1,1,0,1
27503,533066.node129,run_SMD,parallel,2014-04-05 21:12:25,2014-04-05 21:12:26,2014-04-05 21:12:31,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,10.431131,True,0.0,0,0,1,1,0,1
27510,533073.node129,run_SMD,parallel,2014-04-05 21:13:07,2014-04-05 21:13:08,2014-04-05 21:13:14,szia0000@node049.eurora.cineca.it,1,2,8388608,...,0.0,10.432030,True,0.0,0,0,1,1,0,1


In [26]:
outfile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"

In [27]:
jobs.to_csv(outfile_jobs + ".csv")